In [1]:
import pandas as pd
from pyspark import SparkContext
from pyspark.sql.types import StructType
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder\
	.config('spark.jars.packages', 'neo4j-contrib:neo4j-connector-apache-spark_2.12:4.0.1_for_spark_3')\
	.master("local").appName("spark_test").getOrCreate()

22/10/13 10:07:28 WARN Utils: Your hostname, ddi-nos resolves to a loopback address: 127.0.1.1; using 192.168.18.222 instead (on interface ens160)
22/10/13 10:07:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ddi/.ivy2/cache
The jars for the packages stored in: /home/ddi/.ivy2/jars
neo4j-contrib#neo4j-connector-apache-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-67dd0a1b-8c26-4c6d-be1a-b32d85ec1b6d;1.0
	confs: [default]
	found neo4j-contrib#neo4j-connector-apache-spark_2.12;4.0.1_for_spark_3 in spark-packages
:: resolution report :: resolve 674ms :: artifacts dl 20ms
	:: modules in use:
	neo4j-contrib#neo4j-connector-apache-spark_2.12;4.0.1_for_spark_3 from spark-packages in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: 

22/10/13 10:07:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/13 10:07:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/13 10:07:36 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [12]:
booksSchema = StructType() \
                    	.add("cif", LongType())\
                    	.add("account_no", StringType())\
                    	.add("open_date", StringType())

In [13]:
account=spark.read.csv("hdfs://192.168.18.222:8020/user/hive/warehouse/account/account.csv", sep = ';', schema=booksSchema)
account.show(5)

+-----------+-----------+----------+
|        cif| account_no| open_date|
+-----------+-----------+----------+
|18346875234|  442222452|26-07-2017|
|18752032720|13335322450|21-12-2021|
|18752032720|13335322472|26-06-2013|
|18806746420| 2237072704|19-03-2021|
|18806746420|12015201770|16-07-2018|
+-----------+-----------+----------+
only showing top 5 rows



In [16]:
df_test = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://192.168.18.193:7687") \
  .option("labels", "Customer") \
  .option("authentication.basic.username", "neo4j") \
  .option("authentication.basic.password", "ddi123") \
  .load()
df_test.show(5)

+----+----------+---------+---------+--------+-----------+--------------------+
|<id>|  <labels>|     type|     name|    city|        cif|             address|
+----+----------+---------+---------+--------+-----------+--------------------+
|   0|[Customer]|WHOLESALE|     Layo| Jakarta|18346875234|89012 Autumn Leaf...|
|   1|[Customer]|WHOLESALE| Topdrive|Surabaya|18752032720|       83 Chive Pass|
|   2|[Customer]|WHOLESALE|    Meetz| Bandung|18806746420|9411 Schmedeman C...|
|   3|[Customer]|WHOLESALE|     Vipe|  Bekasi|20476940876|4143 Daystar Terrace|
|   4|[Customer]|WHOLESALE|Buzzshare|   Medan|19168353628|       6 Ohio Avenue|
+----+----------+---------+---------+--------+-----------+--------------------+
only showing top 5 rows



In [22]:
account.write \
  .format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://192.168.18.193:7687") \
  .option("database", "testing") \
  .mode("append") \
  .option("authentication.basic.username", "neo4j") \
  .option("authentication.basic.password", "ddi123") \
  .option("labels", ":Account") \
  .save()